# Key untils functions breakdown

MPNN is based on GNN model <br>
We need to define nodes, neighbours and edges



## Gerate training  validation and testing dataset
train and test ID are saved into two text files<br>
Based on cut_off(date, resolution), use list.csv as lookup table to extract <br>
chainID and Hash ID used a format as train['cluster_Id']=[CHAINID, HASH]


In [6]:
import csv
from dateutil import parser
data_path="/home/yunyao/MPNN_breakdown/pdb_2021aug02_sample"
params = {
    "LIST"    : f"{data_path}/list.csv", 
    "VAL"     : f"{data_path}/valid_clusters.txt",
    "TEST"    : f"{data_path}/test_clusters.txt",
    "DIR"     : f"{data_path}",
    "DATCUT"  : "2030-Jan-01",
    "RESCUT"  : 4,      #args.rescut, #resolution cutoff for PDBs
    "HOMO"    : 0.70 #min seq.id. to detect homo chains
}

def build_training_clusters(params, debug):
    val_ids = set([int(l) for l in open(params['VAL']).readlines()])
    test_ids = set([int(l) for l in open(params['TEST']).readlines()])
   
    if debug:
        val_ids = []
        test_ids = []
 
    # read & clean list.csv
    with open(params['LIST'], 'r') as f:
        reader = csv.reader(f)
        next(reader) #skip header
        #CHAINID,DEPOSITION,RESOLUTION,HASH,CLUSTER,SEQUENCE
        rows = [[r[0],r[3],int(r[4])] for r in reader
                if float(r[2])<=params['RESCUT'] and
                parser.parse(r[1])<=parser.parse(params['DATCUT'])]
    
    # compile training and validation sets
    train = {}
    valid = {}
    test = {}

    # example 
    #train['cluster_Id']=[CHAINID, HASH]

    if debug:
        rows = rows[:20]
    for r in rows:  # r[2] is the cluster number 
        if r[2] in val_ids:  
            if r[2] in valid.keys():
                valid[r[2]].append(r[:2])
            else:
                valid[r[2]] = [r[:2]]
        elif r[2] in test_ids:
            if r[2] in test.keys():
                test[r[2]].append(r[:2])
            else:
                test[r[2]] = [r[:2]]
        else:
            if r[2] in train.keys():
                train[r[2]].append(r[:2])
            else:
                train[r[2]] = [r[:2]]
    if debug:
        valid=train       
    return train, valid, test

train, valid, test=build_training_clusters(params,debug=False)

In [10]:
print(len(train.keys()))
print(train[0])

24311
[['2fqm_A', '028171'], ['2fqm_B', '028171'], ['2fqm_C', '028171'], ['2fqm_D', '028171'], ['2fqm_E', '028171'], ['2fqm_F', '028171']]


# CREAT pytorch dataset class 
intialize with ID (train.keys) <br>
loader: loader function, input:  PBDID_chainID '2fqm_C' ,param['Directory']:<br>
generate path pointing to local *.pt file <br>
output:  return {'seq'    : seq, <br>
            'xyz'    : torch.cat(xyz,dim=0),  
            'idx'    : torch.cat(idx,dim=0),  
            'masked' : torch.Tensor(masked).int(),  
            'label'  : item[0]}


In [7]:
import numpy as np
import torch
class PDB_dataset(torch.utils.data.Dataset):
    def __init__(self, IDs, loader, train_dict, params): 
    # loader is a loader function
        self.IDs = IDs
        self.train_dict = train_dict
        self.loader = loader
        self.params = params

    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, index):
        ID = self.IDs[index]
        sel_idx = np.random.randint(0, len(self.train_dict[ID])) # select random chain within the ID
        out = self.loader(self.train_dict[ID][sel_idx], self.params)
        return out

In [27]:
import os

def loader_pdb(item,params):

    pdbid,chid = item[0].split('_')
    PREFIX = "%s/pdb/%s/%s"%(params['DIR'],pdbid[1:3],pdbid)
    print(PREFIX)
    # load metadata
    if not os.path.isfile(PREFIX+".pt"):
        return {'seq': np.zeros(5)}
    meta = torch.load(PREFIX+".pt")
    asmb_ids = meta['asmb_ids']  
    asmb_chains = meta['asmb_chains']
    chids = np.array(meta['chains'])

    # find candidate assemblies which contain chid chain
    asmb_candidates = set([a for a,b in zip(asmb_ids,asmb_chains)
                           if chid in b.split(',')])

    # if the chains is missing is missing from all the assemblies
    # then return this chain alone
    if len(asmb_candidates)<1:
        chain = torch.load("%s_%s.pt"%(PREFIX,chid))
        L = len(chain['seq'])
        return {'seq'    : chain['seq'],
                'xyz'    : chain['xyz'],
                'idx'    : torch.zeros(L).int(),
                'masked' : torch.Tensor([0]).int(),
                'label'  : item[0]}

    # randomly pick one assembly from candidates
    asmb_i = random.sample(list(asmb_candidates), 1)

    # indices of selected transforms
    idx = np.where(np.array(asmb_ids)==asmb_i)[0]

    # load relevant chains in the assembly
    chains = {c:torch.load("%s_%s.pt"%(PREFIX,c))
              for i in idx for c in asmb_chains[i]
              if c in meta['chains']}

    # generate assembly
    asmb = {}
    for k in idx:

        # pick k-th xform
        xform = meta['asmb_xform%d'%k]
        u = xform[:,:3,:3]
        r = xform[:,:3,3]

        # select chains which k-th xform should be applied to
        s1 = set(meta['chains'])
        s2 = set(asmb_chains[k].split(','))
        chains_k = s1&s2

        # transform selected chains 
        for c in chains_k:
            try:
                xyz = chains[c]['xyz']
                xyz_ru = torch.einsum('bij,raj->brai', u, xyz) + r[:,None,None,:]
                asmb.update({(c,k,i):xyz_i for i,xyz_i in enumerate(xyz_ru)})
            except KeyError:
                return {'seq': np.zeros(5)}

    # select chains which share considerable similarity to chid
    seqid = meta['tm'][chids==chid][0,:,1]
    homo = set([ch_j for seqid_j,ch_j in zip(seqid,chids)
                if seqid_j>params['HOMO']])
    # stack all chains in the assembly together
    seq,xyz,idx,masked = "",[],[],[]
    seq_list = []
    for counter,(k,v) in enumerate(asmb.items()):
        seq += chains[k[0]]['seq']
        seq_list.append(chains[k[0]]['seq'])
        xyz.append(v)
        idx.append(torch.full((v.shape[0],),counter))
        if k[0] in homo:
            masked.append(counter)

    return {'seq'    : seq,
            'xyz'    : torch.cat(xyz,dim=0),
            'idx'    : torch.cat(idx,dim=0),
            'masked' : torch.Tensor(masked).int(),
            'label'  : item[0]}

In [8]:
#convert train valid test id into dataloader
train_set = PDB_dataset(list(train.keys()), loader_pdb, train, params)

LOAD_PARAM = {'batch_size': 1,
                'shuffle': True,
                'pin_memory':False,
                'num_workers': 4}
train_loader = torch.utils.data.DataLoader(train_set, **LOAD_PARAM)

In [28]:
train_set.__len__()
print(train.keys())

train[12123][1]

loader_pdb(train[12123][1],params)

dict_keys([12123, 21385, 22021, 10436, 15158, 1824, 4285, 1694, 15132, 7647, 1421, 6879, 4670, 25816, 7600, 18849, 8924, 1671, 10424, 25448, 6118, 20439, 17844, 13097, 7938, 26075, 13459, 11712, 5471, 4121, 9689, 14957, 9986, 3501, 10001, 18346, 3629, 13177, 2780, 11629, 24435, 17981, 3231, 5880, 24382, 24295, 13642, 9620, 16080, 19381, 1835, 24799, 12030, 13488, 11611, 2975, 305, 22535, 11897, 14144, 3107, 22700, 12001, 25826, 13990, 3328, 4446, 7884, 2988, 8667, 23670, 15965, 19561, 15784, 2981, 27582, 10048, 14175, 21865, 25737, 27426, 25268, 16338, 8840, 2159, 23135, 12802, 20193, 2520, 7068, 5051, 13767, 2921, 21101, 27404, 4739, 24250, 14138, 1977, 10354, 17396, 24276, 17010, 23935, 10306, 3661, 22107, 2730, 409, 21310, 2189, 18492, 9469, 16526, 23836, 13115, 8052, 19527, 3459, 52, 22390, 15299, 18897, 26182, 27699, 8149, 4079, 14914, 25396, 3856, 2546, 18932, 25154, 10488, 25325, 16332, 2489, 23286, 4913, 290, 27173, 5231, 1670, 8120, 4999, 15529, 8531, 21662, 12647, 22370, 1319

{'seq': array([0., 0., 0., 0., 0.])}

## Neighour and edge
algorithm: K-nn
